In [1]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow import keras
from keras import layers
import matplotlib.pyplot as plt
from tabulate import tabulate
import joblib
from sklearn.svm import SVC
import pickle

In [2]:
data = pd.read_csv('D:\Barath Suresh Docs\PROGRAMMING\MACHINE LEARNING\credit_card_fraud_detection\creditcard.csv')


In [3]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [4]:
scaler = StandardScaler()
data['Amount'] = scaler.fit_transform(data['Amount'].values.reshape(-1, 1))
data['Time'] = scaler.fit_transform(data['Time'].values.reshape(-1, 1))

In [5]:
X_train_cnn = (X_train - X_train.mean()) / X_train.std()
X_test_cnn = (X_test - X_train.mean()) / X_train.std()

X_train_cnn = X_train_cnn.values.reshape(-1, 6, 5, 1)
X_test_cnn = X_test_cnn.values.reshape(-1, 6, 5, 1)

In [6]:
svm = SVC(kernel='linear', probability=True)
svm.fit(X_train, y_train)

SVC(kernel='linear', probability=True)

In [7]:
reshaped_weight_svm = svm.coef_[0][:25].reshape(25,1)
#print(svm.coef_[0][:25])

In [8]:
model_20 = keras.Sequential([
    keras.layers.Conv2D(filters=32, kernel_size=(2, 2), activation="relu", input_shape=(6, 5, 1)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    
    keras.layers.Conv2D(filters=64, kernel_size=(2, 2), activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),

    keras.layers.Conv2D(filters=64, kernel_size=(2, 2), activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),

    keras.layers.Conv2D(filters=64, kernel_size=(2, 2), activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.25),

    keras.layers.GlobalMaxPooling2D(),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.5),
    
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.5), 

    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(25, activation="relu"),
    keras.layers.Dense(1, activation='softmax', weights=[reshaped_weight_svm, svm.intercept_]),
    keras.layers.Dense(2, activation='softmax')
])

In [9]:
for layer in model_20.layers:
    layer.trainable = False

In [10]:
model_20.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
history = model_20.fit(X_train_cnn, keras.utils.to_categorical(y_train, 2), epochs=10, validation_split=0.2)


Epoch 1/10
5697/5697 [==============================] - 24s 4ms/step - loss: 0.4796 - accuracy: 0.9983 - val_loss: 0.4797 - val_accuracy: 0.9982
Epoch 2/10
5697/5697 [==============================] - 22s 4ms/step - loss: 0.4796 - accuracy: 0.9983 - val_loss: 0.4797 - val_accuracy: 0.9982
Epoch 3/10
5697/5697 [==============================] - 24s 4ms/step - loss: 0.4796 - accuracy: 0.9983 - val_loss: 0.4797 - val_accuracy: 0.9982
Epoch 4/10
5697/5697 [==============================] - 23s 4ms/step - loss: 0.4796 - accuracy: 0.9983 - val_loss: 0.4797 - val_accuracy: 0.9982
Epoch 5/10
5697/5697 [==============================] - 23s 4ms/step - loss: 0.4796 - accuracy: 0.9983 - val_loss: 0.4797 - val_accuracy: 0.9982
Epoch 6/10
5697/5697 [==============================] - 24s 4ms/step - loss: 0.4796 - accuracy: 0.9983 - val_loss: 0.4797 - val_accuracy: 0.9982
Epoch 7/10
5697/5697 [==============================] - 22s 4ms/step - loss: 0.4796 - accuracy: 0.9983 - val_loss: 0.4797 - val_ac

In [12]:
test_loss, test_acc = model_20.evaluate(X_test_cnn, keras.utils.to_categorical(y_test, 2))
print('Test accuracy:',test_acc*100,'%')

1781/1781 [==============================] - 3s 2ms/step - loss: 0.4796 - accuracy: 0.9982
Test accuracy: 99.82268810272217 %
